# Reading GHCN files

Code adapated from https://gitlab.com/snippets/1838910

Data downloaded from ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/

In [1]:
import os
import pandas as pd

In [2]:
# Metadata specs #

metadata_col_specs = [
    (0,  12),
    (12, 21),
    (21, 31),
    (31, 38),
    (38, 41),
    (41, 72),
    (72, 76),
    (76, 80),
    (80, 86)
]

metadata_names = [
    "ID",
    "LATITUDE",
    "LONGITUDE",
    "ELEVATION",
    "STATE",
    "NAME",
    "GSN FLAG",
    "HCN/CRN FLAG",
    "WMO ID"]

metadata_dtype = {
    "ID": str,
    "STATE": str,
    "NAME": str,
    "GSN FLAG": str,
    "HCN/CRN FLAG": str,
    "WMO ID": str
    }


# Data specs #

data_header_names = [
    "ID",
    "YEAR",
    "MONTH",
    "ELEMENT"]

data_header_col_specs = [
    (0,  11),
    (11, 15),
    (15, 17),
    (17, 21)]

data_header_dtypes = {
    "ID": str,
    "YEAR": int,
    "MONTH": int,
    "ELEMENT": str}

data_col_names = [[
    "VALUE" + str(i + 1),
    "MFLAG" + str(i + 1),
    "QFLAG" + str(i + 1),
    "SFLAG" + str(i + 1)]
    for i in range(31)]

# Join sub-lists
data_col_names = sum(data_col_names, [])

data_replacement_col_names = [[
    ("VALUE", i + 1),
    ("MFLAG", i + 1),
    ("QFLAG", i + 1),
    ("SFLAG", i + 1)]
    for i in range(31)]
# Join sub-lists
data_replacement_col_names = sum(data_replacement_col_names, [])
data_replacement_col_names = pd.MultiIndex.from_tuples(
    data_replacement_col_names,
    names=['VAR_TYPE', 'DAY'])

data_col_specs = [[
    (21 + i * 8, 26 + i * 8),
    (26 + i * 8, 27 + i * 8),
    (27 + i * 8, 28 + i * 8),
    (28 + i * 8, 29 + i * 8)]
    for i in range(31)]
data_col_specs = sum(data_col_specs, [])

data_col_dtypes = [{
    "VALUE" + str(i + 1): int,
    "MFLAG" + str(i + 1): str,
    "QFLAG" + str(i + 1): str,
    "SFLAG" + str(i + 1): str}
    for i in range(31)]
data_header_dtypes.update({k: v for d in data_col_dtypes for k, v in d.items()})


## Read stations.txt and convert to xls

In [ ]:
def read_station_metadata(filename):
    """Reads in station metadata

    :filename: ghcnd station metadata file.
    :returns: station metadata as a pandas Dataframe

    """
    df = pd.read_fwf(filename, metadata_col_specs, names=metadata_names,
                      dtype=metadata_dtype)

    return df


In [ ]:
df = read_station_metadata('data/ghcnd-stations.txt')

In [ ]:
output_dir = 'output'
output_filename = 'ghcnd-stations.xlsx'
output_path = os.path.join(output_dir, output_filename)
df.to_excel(output_path)

In [ ]:
filename = 'data/2020.csv'
data_col_dtypes = {
    'ID': str,
    'DATE': str,
    'ELEMENT': str,
    'DATA VALUE': int,
    'M-FLAG': str,
    'Q-FLAG': str,
    'S-FLAG': str,
    'OBS-TIME': str}


column_names = [
    'ID','DATE','ELEMENT','DATA VALUE','M-FLAG','Q-FLAG','S-FLAG','OBS-TIME'
]

ghcn_all = pd.read_csv(filename, names=column_names, dtype=data_col_dtypes)
ghcn_all

In [ ]:
stations = read_station_metadata()
wisc_stations = stations[stations['STATE'] == 'WI']
wisc_stations_list = wisc_stations['ID'].unique().tolist()

In [ ]:
filtered = ghcn_all[(ghcn_all['ELEMENT'] == 'SNOW') & (ghcn_all['DATE'].str.startswith('202001'))]
filtered

In [ ]:
for station in wisc_stations_list:
    station_df = filtered[filtered['ID'] == station]
    if not station_df.empty:
        output_file = 'output/wi/{}.xlsx'.format(station)
        station_df.to_excel(output_file, index=False)

# Process .dly files

for x in *.xlsx; do wget ftp://ftp.ncdc.noaa.gov/pub/data/ghcn/daily/all/${x%%.*}.dly -P dly;done

In [3]:
def read_ghcn_data_file(filename):
    df = pd.read_fwf(
        filename,
        colspecs=data_header_col_specs + data_col_specs,
        names=data_header_names + data_col_names,
        dtype=data_header_dtypes
        )
    return df

In [16]:
files = os.listdir('output/wi/dly/')
dfs = []
for file in files:
    path = os.path.join('output', 'wi', 'dly', file)
    df = read_ghcn_data_file(path)
    dfs.append(df)
    
df = pd.concat(dfs, axis=0, ignore_index=True)


In [17]:
df_jan = df[(df['YEAR'] == 2020) & (df['MONTH'] == 1) & (df['ELEMENT'] == 'SNOW')]
df_jan

,ID,YEAR,MONTH,ELEMENT,VALUE1,MFLAG1,QFLAG1,SFLAG1,VALUE2,MFLAG2,...,QFLAG29,SFLAG29,VALUE30,MFLAG30,QFLAG30,SFLAG30,VALUE31,MFLAG31,QFLAG31,SFLAG31
923,USC00477964,2020,1,SNOW,0,NaN,NaN,7,0,NaN,...,NaN,7,0,NaN,NaN,7,0,NaN,NaN,7
1067,US1WIWK0047,2020,1,SNOW,0,T,NaN,N,0,NaN,...,NaN,N,0,NaN,NaN,N,0,T,NaN,N
2851,USC00478316,2020,1,SNOW,0,NaN,NaN,7,0,NaN,...,NaN,7,0,T,NaN,7,5,NaN,NaN,7
3007,US1WIDA0046,2020,1,SNOW,0,T,NaN,N,-9999,NaN,...,NaN,N,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN
3410,USC00471681,2020,1,SNOW,0,NaN,NaN,7,0,NaN,...,NaN,7,0,NaN,NaN,7,0,NaN,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948526,US1WIPK0007,2020,1,SNOW,0,NaN,NaN,N,0,NaN,...,NaN,N,0,NaN,NaN,N,0,T,NaN,N
954837,USC00473058,2020,1,SNOW,0,NaN,NaN,7,0,NaN,...,NaN,7,0,NaN,NaN,7,0,T,NaN,7
960683,USC00471897,2020,1,SNOW,0,NaN,NaN,7,0,NaN,...,NaN,7,0,NaN,NaN,7,0,T,NaN,7
960835,US1WIDA0063,2020,1,SNOW,0,NaN,NaN,N,0,NaN,...,NaN,N,-9999,NaN,NaN,NaN,-9999,NaN,NaN,NaN


In [19]:
df_jan = df_jan.replace(-9999, 0)
df_jan

,ID,YEAR,MONTH,ELEMENT,VALUE1,MFLAG1,QFLAG1,SFLAG1,VALUE2,MFLAG2,...,QFLAG29,SFLAG29,VALUE30,MFLAG30,QFLAG30,SFLAG30,VALUE31,MFLAG31,QFLAG31,SFLAG31
923,USC00477964,2020,1,SNOW,0,NaN,NaN,7,0,NaN,...,NaN,7,0,NaN,NaN,7,0,NaN,NaN,7
1067,US1WIWK0047,2020,1,SNOW,0,T,NaN,N,0,NaN,...,NaN,N,0,NaN,NaN,N,0,T,NaN,N
2851,USC00478316,2020,1,SNOW,0,NaN,NaN,7,0,NaN,...,NaN,7,0,T,NaN,7,5,NaN,NaN,7
3007,US1WIDA0046,2020,1,SNOW,0,T,NaN,N,0,NaN,...,NaN,N,0,NaN,NaN,NaN,0,NaN,NaN,NaN
3410,USC00471681,2020,1,SNOW,0,NaN,NaN,7,0,NaN,...,NaN,7,0,NaN,NaN,7,0,NaN,NaN,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
948526,US1WIPK0007,2020,1,SNOW,0,NaN,NaN,N,0,NaN,...,NaN,N,0,NaN,NaN,N,0,T,NaN,N
954837,USC00473058,2020,1,SNOW,0,NaN,NaN,7,0,NaN,...,NaN,7,0,NaN,NaN,7,0,T,NaN,7
960683,USC00471897,2020,1,SNOW,0,NaN,NaN,7,0,NaN,...,NaN,7,0,NaN,NaN,7,0,T,NaN,7
960835,US1WIDA0063,2020,1,SNOW,0,NaN,NaN,N,0,NaN,...,NaN,N,0,NaN,NaN,NaN,0,NaN,NaN,NaN


In [20]:
output_file = 'output/wi_dly_jan2020.xlsx'
df_jan.to_excel(output_file, index=False)